In [14]:
from keras import backend as K

def dice_coefficient(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coefficient_loss(y_true, y_pred):
    return -dice_coefficient(y_true, y_pred)


def weighted_dice_coefficient(y_true, y_pred, axis=(-3, -2, -1), smooth=0.00001):
    """
    Weighted dice coefficient. Default axis assumes a "channels first" data structure
    :param smooth:
    :param y_true:
    :param y_pred:
    :param axis:
    :return:
    """
    return K.mean(2. * (K.sum(y_true * y_pred,
                              axis=axis) + smooth/2)/(K.sum(y_true,
                                                            axis=axis) + K.sum(y_pred,
                                                                               axis=axis) + smooth))


def weighted_dice_coefficient_loss(y_true, y_pred):
    return -weighted_dice_coefficient(y_true, y_pred)


def label_wise_dice_coefficient(y_true, y_pred, label_index):
    return dice_coefficient(y_true[:, label_index], y_pred[:, label_index])


def get_label_dice_coefficient_function(label_index):
    f = partial(label_wise_dice_coefficient, label_index=label_index)
    f.__setattr__('__name__', 'label_{0}_dice_coef'.format(label_index))
    return f


dice_coef = dice_coefficient
dice_coef_loss = dice_coefficient_loss

In [15]:
import numpy as np
import keras
import nibabel as nib
import pickle
class SubDataGenerator(keras.utils.data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=1, dim=(240,240,155), n_channels=4,
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = None
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self.__data_generation(list_IDs_temp)

        return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_channels, *self.dim))
        y1 = np.empty((self.batch_size, 3, *self.dim))
        y2 = np.empty(self.batch_size)

        # Generate data
        # Decode and load the data
        for i, ID in enumerate(list_IDs_temp):

            # 1) the "enhancing tumor" (ET), 2) the "tumor core" (TC), and 3) the "whole tumor" (WT) 
            # The ET is described by areas that show hyper-intensity in T1Gd when compared to T1, but also when compared to “healthy” white matter in T1Gd. The TC describes the bulk of the tumor, which is what is typically resected. The TC entails the ET, as well as the necrotic (fluid-filled) and the non-enhancing (solid) parts of the tumor. The appearance of the necrotic (NCR) and the non-enhancing (NET) tumor core is typically hypo-intense in T1-Gd when compared to T1. The WT describes the complete extent of the disease, as it entails the TC and the peritumoral edema (ED), which is typically depicted by hyper-intense signal in FLAIR.
            # The labels in the provided data are: 
            # 1 for NCR & NET (necrotic (NCR) and the non-enhancing (NET) tumor core) = TC ("tumor core")
            # 2 for ED ("peritumoral edema")
            # 4 for ET ("enhancing tumor")
            # 0 for everything else

            X[i,] = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(ID), "rb" ) )

        return X

In [16]:
import numpy as np
from nilearn.image.image import check_niimg
from nilearn.image.image import _crop_img_to as crop_img_to


def crop_img(img, rtol=1e-8, copy=True, return_slices=False):
    """Crops img as much as possible
    Will crop img, removing as many zero entries as possible
    without touching non-zero entries. Will leave one voxel of
    zero padding around the obtained non-zero area in order to
    avoid sampling issues later on.
    Parameters
    ----------
    img: Niimg-like object
        See http://nilearn.github.io/manipulating_images/input_output.html
        img to be cropped.
    rtol: float
        relative tolerance (with respect to maximal absolute
        value of the image), under which values are considered
        negligeable and thus croppable.
    copy: boolean
        Specifies whether cropped data is copied or not.
    return_slices: boolean
        If True, the slices that define the cropped image will be returned.
    Returns
    -------
    cropped_img: image
        Cropped version of the input image
    """

    img = check_niimg(img)
    data = img.get_data()
    infinity_norm = max(-data.min(), data.max())
    passes_threshold = np.logical_or(data < -rtol * infinity_norm,
                                     data > rtol * infinity_norm)

    if data.ndim == 4:
        passes_threshold = np.any(passes_threshold, axis=-1)
    coords = np.array(np.where(passes_threshold))
    start = coords.min(axis=1)
    end = coords.max(axis=1) + 1

    # pad with one voxel to avoid resampling problems
    start = np.maximum(start - 1, 0)
    end = np.minimum(end + 1, data.shape[:3])

    slices = [slice(s, e) for s, e in zip(start, end)]

    if return_slices:
        return slices

    return crop_img_to(img, slices, copy=copy)

In [17]:
from keras.models import load_model
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

model = load_model('model3_new.h5', custom_objects={'InstanceNormalization': InstanceNormalization(axis=1),
                                                      'weighted_dice_coefficient_loss': weighted_dice_coefficient_loss,
                                                      'weighted_dice_coefficient' : weighted_dice_coefficient,
                                                       'dice_coefficient' : dice_coefficient,
                                                      })

In [18]:
# import os
# import pickle
# directories = os.listdir("./dataCorrected")
# params = {'dim': (160,192,160),
#     'batch_size': 1,
#     'n_classes': 3,
#     'n_channels': 4,
#     'shuffle': False}
# for d in directories:
#     validation_generator = SubDataGenerator([d], **params)
#     predictions_1_pred = model.predict_generator(generator=validation_generator)
#     pickle.dump(predictions_1_pred[0][0], open( "D:\segPredictionsPickle/%s_seg_mask_3ch.pkl"%(d), "wb" ) )
# #     nib_img = nib.Nifti1Image(predictions_1_pred[0][0], affine=np.eye(4))
# #     nib.save(nib_img, f'D:\segPredictions/{d}_predicted.nii.gz')

C:\Users\HassanIslam\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [ ]:
params = {'dim': (160,192,160),
    'batch_size': 1,
    'n_classes': 3,
    'n_channels': 4,
    'shuffle': False}
validation_generator = SubDataGenerator(['BraTS19_CBICA_AAB_1'], **params)
predictions_1_pred = model.predict_generator(generator=validation_generator)


In [ ]:
predictions_1_pred[0][0][1].shape
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

plt.imshow(predictions_1_pred[0][0][2][:,:,70],aspect="auto")
plt.show()

In [ ]:
ID = 'BraTS19_CBICA_AAB_1'
gt = pickle.load( open( "D:\croppedFiles/%s_seg_mask_3ch.pkl"%(ID), "rb" ) )
print(gt.shape)
nib_img = nib.Nifti1Image(predictions_1_pred[0], affine=np.eye(4))
plt.imshow(gt[2][:,:,70],aspect="auto")
plt.show()

In [ ]:
import numpy as np
import copy
import nibabel as nib

from tifffile import imsave
# from libtiff import TIFF

# from skimage.io._plugins import freeimage_plugin as fi

ID = 'BraTS19_CBICA_AAB_1'
img1 = './dataCorrected/' + ID + '/' + ID + '_flair.nii.gz'
img2 = './dataCorrected/' + ID + '/' + ID + '_t1.nii.gz'
img3 = './dataCorrected/' + ID + '/' + ID + '_t1ce.nii.gz'
img4 = './dataCorrected/' + ID + '/' + ID + '_t2.nii.gz'
img5 = './data/' + ID + '/' + ID + '_seg.nii.gz'

newimage = nib.concat_images([img1, img2, img3, img4, img5])

cropped = crop_img(newimage)
print(cropped.shape)
# img_array = np.array(cropped.dataobj)
# z = np.rollaxis(img_array, 3, 0)
# padded_image = np.zeros((5, 160, 192, 160))
# padded_image[:z.shape[0], :z.shape[1], :z.shape[2], :z.shape[3]] = z

# a, b, c, d, seg_mask = np.split(padded_image, 5, axis=0)


# images = np.concatenate([a, b, c, d], axis=0)

# print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

# # split the channels:
# # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))
# seg_mask_1 = np.zeros((1, 160, 192, 160))
# seg_mask_1[seg_mask.astype(int) > 0] = 1
# seg_mask_2 = np.zeros((1, 160, 192, 160))
# seg_mask_2[seg_mask.astype(int) > 1] = 1
# seg_mask_3 = np.zeros((1, 160, 192, 160))
# seg_mask_3[seg_mask.astype(int) > 2] = 1
# seg_mask_3ch = np.concatenate([seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype(int)

# # def scale_image(image_array):
# #     image_array = image_array.astype(float)
# #     image_array *= 255.0/image_array.max() # convert to 8-bit pixel values
# #     image_array = image_array.astype(int)
# #     return image_array

# # img_array_list = [a,seg_mask_1,seg_mask_2,seg_mask_3]
# # for img_array in img_array_list:
# #     img_array = scale_image(img_array)

# a = a.astype(float)
# a *= 255.0/a.max()  # convert to 8-bit pixel values
# a = np.rollaxis(a, 0, 2) # cxyz -> xycz for imagej
# a = np.rollaxis(a, 0, 3) # switching x and z
# a = a.astype('uint8')
# #     print("unique flair values:", np.unique(a))

# seg_mask_1 = seg_mask_1.astype(float)
# seg_mask_1 *= 255.0/seg_mask_1.max()  # convert to 8-bit pixel values
# seg_mask_1 = np.rollaxis(seg_mask_1, 0, 2) # cxyz -> xycz for imagej
# seg_mask_1 = np.rollaxis(seg_mask_1, 0, 3) # switching x and z
# seg_mask_1 = seg_mask_1.astype('uint8')
# #     print("unique segment mask values:", np.unique(seg_mask_1))

# seg_mask_2 = seg_mask_2.astype(float)
# seg_mask_2 *= 255.0/seg_mask_2.max()  # convert to 8-bit pixel values
# seg_mask_2 = np.rollaxis(seg_mask_2, 0, 2) # cxyz -> xycz for imagej
# seg_mask_2 = np.rollaxis(seg_mask_2, 0, 3) # switching x and z
# seg_mask_2 = seg_mask_2.astype('uint8')

# seg_mask_3 = seg_mask_3.astype(float)
# seg_mask_3 *= 255.0/seg_mask_3.max()  # convert to 8-bit pixel values
# seg_mask_3 = np.rollaxis(seg_mask_3, 0, 2) # cxyz -> xycz for imagej
# seg_mask_3 = np.rollaxis(seg_mask_3, 0, 3) # switching x and z
# seg_mask_3 = seg_mask_3.astype('uint8')

# #     ground_truth = np.concatenate(
# #         [a, seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype('uint8')

# #     print("unique flair + segment mask values:", np.unique(ground_truth))
# # #     shape.ground_truth
# #     flairimg = flairimg.astype(float)
# #     flairimg *= 255.0/flairimg.max() # convert to 8-bit pixel values
# #     flairimg = flairimg.astype(int)
# #     print(np.unique(flairimg))
# #     print("final image shape:", ground_truth.shape)
# #     imsave("./channel_split/"+testIDlist[i]+"ground_truth.tif", ground_truth, 'imagej')
    
    
# a_img = nib.Nifti1Image(a, affine=np.eye(4))
# nib.save(a_img, './test/BraTS19_CBICA_AAB_1_a.nii.gz')
# s1_img = nib.Nifti1Image(seg_mask_1, affine=np.eye(4))
# nib.save(s1_img, './test/BraTS19_CBICA_AAB_1_s1.nii.gz')
# s2_img = nib.Nifti1Image(seg_mask_2, affine=np.eye(4))
# nib.save(s2_img, './test/BraTS19_CBICA_AAB_1_s2.nii.gz')
# s3_img = nib.Nifti1Image(seg_mask_3, affine=np.eye(4))
# nib.save(s3_img, './test/BraTS19_CBICA_AAB_1_s3.nii.gz')
# imsave("./test/"+ID+"_flair.tif", a, 'imagej')
# imsave("./test/"+ID+"_ground_truth_1.tif", seg_mask_1, 'imagej')
# imsave("./test/"+ID+"_ground_truth_2.tif", seg_mask_2, 'imagej')
# imsave("./test/"+ID+"_ground_truth_3.tif", seg_mask_3, 'imagej')

# # imarray = pickle.load(open( "./channel_split/prediction_"+str(i)+".pkl", "rb" ) )
# imarray = predictions_1_pred[0][0]

# prediction_thresh = copy.deepcopy(imarray)
# prediction_thresh[prediction_thresh < 0.5] = 0.
# prediction_thresh[prediction_thresh >= 0.5] = 1.
# prediction_thresh = prediction_thresh
# print(np.unique(prediction_thresh))
# prediction_thresh *= 255.0/prediction_thresh.max() # convert to 8-bit pixel values
# prediction_thresh = prediction_thresh.astype('uint8')
# prediction_thresh = np.rollaxis(prediction_thresh, 1, 3) # switching x and z; c will be taken care of in split
# print(np.unique(prediction_thresh))
# print(prediction_thresh.shape)

# pred1, pred2, pred3 = np.split(prediction_thresh, 3, axis=0)

# imsave("./test/"+ID+"_predicted_1.tif", pred1, 'imagej')
# imsave("./test/"+ID+"_predicted_2.tif", pred2, 'imagej')
# imsave("./test/"+ID+"_predicted_3.tif", pred3, 'imagej')

# #     # print("images shape:", images.shape, "images values:", np.unique(images.astype(int)))

# #     # split the channels:
# #     # seg_mask_1 = copy.deepcopy(seg_mask.astype(int))


# #     #     seg_mask_3ch = np.concatenate(
# # #         [seg_mask_1, seg_mask_2, seg_mask_3], axis=0).astype(int)

# # #     imarray *= 255.0/imarray.max()

# # #     imsave("./channel_split/"+testIDlist[i]+"ground_truth_1.tif", seg_mask_1, 'imagej')
# # #     imsave("./channel_split/"+testIDlist[i]+"ground_truth_2.tif", seg_mask_2, 'imagej')
# # #     imsave("./channel_split/"+testIDlist[i]+"ground_truth_3.tif", seg_mask_3, 'imagej')

In [ ]:
from tifffile import imsave
imarray1 = predictions_1_pred[0][0][0]
# imarray[]
imarray1 *= 255.0/imarray1.max()
imarray2 = predictions_1_pred[0][0][1]
# imarray[]
imarray2 *= 255.0/imarray2.max()
imarray3 = predictions_1_pred[0][0][2]
# imarray[]
imarray3 *= 255.0/imarray3.max()

X = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(ID), "rb" ) )
imarray1 = imarray1 + X[0]


nib_img = nib.Nifti1Image(imarray1, affine=np.eye(4))
nib.save(nib_img, './test/BraTS19_CBICA_AAB_1_predicted.nii.gz')

# imsave("./test/prediction_full1.tif", imarray1)

#     # make ground truth 
# ID = 'BraTS19_CBICA_AAB_1'
# img1 = './dataCorrected/' + ID + '/' + ID + '_flair.nii.gz'
# img2 = './dataCorrected/' + ID + '/' + ID + '_t1.nii.gz'
# img3 = './dataCorrected/' + ID + '/' + ID + '_t1ce.nii.gz'
# img4 = './dataCorrected/' + ID + '/' + ID + '_t2.nii.gz'
# img5 = './data/' + ID + '/' + ID + '_seg.nii.gz'

# newimage = nib.concat_images([img1, img2, img3, img4, img5])
# cropped = crop_img(newimage)         
# img_array = np.array(cropped.dataobj)
# z = np.rollaxis(img_array, 3, 0)

# padded_image = np.zeros((5,160,192,160))
# padded_image[:z.shape[0],:z.shape[1],:z.shape[2],:z.shape[3]] = z

# a,b,c,d,seg_mask = np.split(padded_image, 5, axis=0)

# images = np.concatenate([a,b,c,d], axis=0)
# imsave("./test/"+ID+"ground_truth_full.tif", images)

In [ ]:
new = np.rollaxis(predictions_1_pred[0][0], 0, 4)  
new.shape

# imarray = new
# imarray *= 255.0/imarray.max()
nib_img = nib.Nifti1Image(new, affine=np.eye(4))
nib.save(nib_img, './test/BraTS19_CBICA_AAB_1_predicted_rolled.nii.gz')

In [ ]:
predictions_1_pred[0][0].shape

In [ ]:
import pickle
from tifffile import imsave
ID = 'BraTS19_CBICA_AAB_1'
sgm = pickle.load( open( "D:\croppedFiles/%s_seg_mask_3ch.pkl"%(ID), "rb" ) )   
sgm = (sgm * 255.0)
imsave("./test/"+ID+"groundtruth_full.tif", sgm)

In [ ]:
predictions_1_pred[0][0].shape

In [ ]:
import os
from radiomics import featureextractor
import pandas as pd
import glob
import csv
import SimpleITK as sitk
ID = 'BraTS19_CBICA_AAB_1'
paramPath = os.path.join(os.getcwd(), "Params.yaml")
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)
image = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(ID), "rb" ) )
keys, values = [], []
modalities = [0,2]
for i in modalities:
    for j in range(3):
        image = sitk.GetImageFromArray(image[i])
        label = sitk.GetImageFromArray(predictions_1_pred[0][0][j])
        result = extractor.execute(image, label)
        for key, value in result.items():
            keys.append(key)
            values.append(value)
        if(i == 2 & j==2):
            keys.append('Age')
            values.append(60)
            
        image = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(ID), "rb" ) )
with open(f"./test/{ID}_features.csv", "w") as outfile:
    csvwriter = csv.writer(outfile)
    csvwriter.writerow(keys)
    csvwriter.writerow(values)
x = pd.read_csv(f"./test/{ID}_features.csv")


In [ ]:
data = pd.read_csv('./features/from_predictions/top_corr_data_20_c.csv')
del data['Survival']
x[data.columns]

In [ ]:
import pickle
model = pickle.load(open('survival_model', 'rb'))

In [ ]:
model.predict(x[data.columns])

In [1]:
import numpy as np
import keras
import nibabel as nib
import pickle
import os
from keras import backend as K
import tensorflow as tf
r = {}
directories = os.listdir("./dataCorrected")
for d in directories:
    true = pickle.load( open( "D:\croppedFiles/%s_seg_mask_3ch.pkl"%(d), "rb" ) )
    pred = nib.load("D:\segPredictions/%s_predicted.nii.gz"%(d))
    pred = np.array(pred.dataobj)
    y_true_f = K.flatten(true)
    y_pred_f = K.flatten(pred)
    y_true_f = tf.cast(y_true_f, tf.float32)
    y_pred_f = tf.cast(y_pred_f, tf.float32)
    intersection = K.sum((y_true_f) * (y_pred_f))
    dice = (2. * intersection + 1.0) / (K.sum(y_true_f)+ K.sum(y_pred_f) + 1.0)
    r[d] = dice

C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\HassanIslam\anaconda3\envs\FYP\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [13]:
r

{'BraTS19_2013_10_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.84374356>,
 'BraTS19_2013_11_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.60655063>,
 'BraTS19_2013_12_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.8352653>,
 'BraTS19_2013_13_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.80278057>,
 'BraTS19_2013_14_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.8327148>,
 'BraTS19_2013_17_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.81670576>,
 'BraTS19_2013_18_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.8704163>,
 'BraTS19_2013_19_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.8105531>,
 'BraTS19_2013_20_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.6539614>,
 'BraTS19_2013_21_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.528967>,
 'BraTS19_2013_22_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.47310317>,
 'BraTS19_2013_23_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.7982266>,
 'BraTS19_2013_25_1': <tf.Tensor: shape=(), dtype=float32, numpy=0.52975

In [ ]:
BraTS19_TCIA06_211_1 0.4203
BraTS19_CBICA_BGG_1 0.9433
BraTS19_CBICA_AOZ_1 0.7477

In [14]:
import pickle
import nibabel as nib
import numpy as np
ID = 'BraTS19_CBICA_AOZ_1'
X = pickle.load( open( "D:\croppedFiles/%s_images.pkl"%(ID), "rb" ) )
Y= pickle.load( open( "D:\croppedFiles/%s_seg_mask_3ch.pkl"%(ID), "rb" ) )
Y = (Y * 255.0)
avg_true_00 = X[0] + Y[2]
nib_img = nib.Nifti1Image(avg_true_00, affine=np.eye(4))
nib.save(nib_img, './test/avg_true_02.nii.gz')